In [4]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shapely.ops as so
import shapely.geometry as sg
import tensorflow.keras.preprocessing.image as tf_image
%matplotlib widget

In [5]:
class Grain(object):
    """ Stores grain information and plot representation """

    def __init__(self, xy:list=[], data:pd.Series=None):
        # Data
        self.xy = xy
        self.data = data

        # Display
        self.normal_props = {
            'alpha': 0.6
        }
        self.selected_props = {
            'alpha': 1.0,
            'facecolor': 'red'
        }
        # self.ax = None
        self.patch = None
        self.selected = False
    
    def make_patch(self, ax):
        # self.ax = ax
        (self.patch,) = ax.fill(
            *self.xy,
            edgecolor='black',
            linewidth=5.0,
            picker=True,
            **self.normal_props
        )
        self.normal_props['facecolor'] = self.patch.get_facecolor()
        return self.patch

    def select(self):
        self.selected = ~self.selected
        props = self.selected_props if self.selected else self.normal_props
        self.patch.set(**props)
        return self.selected


class GrainPlot(object):
    """ Interactive plot to create, delete, and merge grains """

    def __init__(self, grains, image=None, predictor=None):
        # Input
        self.grains = grains
        self.image = image
        self.predictor = predictor
        # Plot
        self.fig = plt.figure(figsize=(6, 4))
        self.canvas = self.fig.canvas
        self.ax = self.fig.add_subplot(aspect='equal')
        self.ax.set_xticks([])
        self.ax.set_yticks([])
        if isinstance(image, np.ndarray):
            self.ax.imshow(image)
        # Interaction events
        self.cids = []
        self.selected_grains = []
        self.left_cursor = patches.Circle((10, 10), radius=5, color='lime', visible=False)
        self.right_cursor = patches.Circle((10, 10), radius=5, color='red', visible=False)
        self.events = {
            'button_press_event': self.onclick,
            'pick_event': self.onpick,
            'key_press_event': self.onpress
        }
        # Draw elements on plot
        with plt.ioff():
            for grain in grains:
                grain.make_patch(self.ax)
            self.ax.add_patch(self.left_cursor)
        self.ax.add_patch(self.right_cursor)
        # self.fig.tight_layout()
        # Seems to help with occasional failure to draw updates
        plt.pause(0.1)

    # Helper functions ---
    def set_cursor(self, cursor, xy):
        if xy:
            cursor.set_center(xy)
            cursor.set_visible(True)
        else:
            cursor.set_visible(False)

    def unselect_cursors(self):
        self.set_cursor(self.left_cursor, False)
        self.set_cursor(self.right_cursor, False)

    def unselect_grains(self):
        for grain in self.selected_grains:
            grain.select()
        self.selected_grains = []

    def unselect_all(self):
        self.unselect_cursors()
        self.unselect_grains()

    # Manage grains ---
    def create_grain(self):
        """ Attempt to find and add grain at selected point """
        # Verify that we've actually selected something
        if not self.left_cursor.get_visible():
            return
        # Two-point prompt (grain and background)
        x1, y1 = self.left_cursor.get_center()
        if self.right_cursor.get_visible():
            x2, y2 = self.right_cursor.get_center()
            x, y = seg.two_point_prompt(x2, y2, x1, y1, image=self.image, predictor=self.predictor)
        # One-point prompt (grain only)
        else:
            x, y, mask = seg.one_point_prompt(x1, y1, image=self.image, predictor=self.predictor)
        # Add grain to plot
        grain = Grain((x, y))
        grain.make_patch(self.ax)
        # Add grain to list
        self.grains.append(grain)
        # Reset cursors
        self.unselect_cursors()

    def delete_grains(self):
        """ Delete all selected grains """
        with plt.ioff():
            for grain in self.selected_grains:
                # Hide grain from plot
                # grain.patch.set_visible(False) # Helps with intermittent failure to update
                # self.ax.draw_artist(grain.patch)
                grain.patch.remove()
                # Delete grain from list
                self.grains.remove(grain)
        # self.canvas.draw()
        self.selected_grains = []

    def merge_grains(self):
        """ Merge all selected grains """
        # Verify there are at least two grains to merge
        if len(self.selected_grains) < 2:
            return
        # Find vertices of merged grains using Shapely
        polys = [sg.Polygon(g.patch.get_path().vertices) for g in self.selected_grains]
        poly = so.unary_union(polys)
        # Verify that grains actually overlap, otherwise reset selection
        if isinstance(poly, sg.MultiPolygon):
            self.unselect_all()
            return
        # Make new grain
        new_grain = Grain(poly.exterior.xy)
        self.grains.append(new_grain)
        new_grain.make_patch(self.ax)
        # Clear old grains
        self.delete_grains()

    # Events ---
    def onclick(self, event):
        """ Handle clicking anywhere on plot """
        # Only process individual clicks, only when no grains selected
        if event.dblclick is True or len(self.selected_grains) > 0:
            return
        # Grain prompt (left click)
        if event.button == 1:
            self.set_cursor(self.left_cursor, (event.xdata, event.ydata))
        # Background prompt (right click)
        elif event.button == 3:
            self.set_cursor(self.right_cursor, (event.xdata, event.ydata))

    def onpick(self, event):
        """ Handle clicking on an existing grain """
        # Only process individual left-clicks
        if event.mouseevent.dblclick or event.mouseevent.button != 1:
            return
        # Hide cursors
        self.unselect_cursors()
        # Add selected grain to list
        for grain in self.grains:
            if event.artist is grain.patch:
                if grain.select():
                    self.selected_grains.append(grain)
                else:
                    self.selected_grains.remove(grain)
                break
    
    def onpress(self, event):
        """ Handle key presses """
        if event.key == 'c':
            self.create_grain()
        elif event.key == 'delete':
            self.delete_grains()
        elif event.key == 'm':
            self.merge_grains()
        elif event.key == 'escape':
            self.unselect_all()
        # # Maybe helps with failure to draw updates?
        else: 
            plt.pause(0.01)
            self.canvas.draw()

    def activate(self):
        """ Enable interactive features """
        for event, handler in self.events.items():
            self.cids.append(self.canvas.mpl_connect(event, handler))

    def deactivate(self):
        """ Disable interactive features """
        for cid in self.cids:
            self.canvas.mpl_disconnect(cid)
        self.cids = []
    
    # Output ---
    def get_mask(self) -> np.ndarray:
        # TODO
        self.fig.savefig('test.png', transparent=True)
        return

    def get_data(self) -> pd.DataFrame:
        return pd.concat([g.data for g in self.grains], axis=1).T


In [ ]:
# # Load image
# fname = './torrey_pines_small.jpeg'
# image = np.array(tf_image.load_img(fname))

# Make grains
grains = []
centroids = [(0, 0), (1, 0), (0, 1), (1, 1)]
for c in centroids:
    x, y = c
    x = [x-0.5, x+0.5, x+0.5, x-0.5]
    y = [y+0.5, y+0.5, y-0.5, y-0.5]
    grains.append(Grain([x, y]))

# Plot grains
grain_plot = GrainPlot(grains, image=image)
grain_plot.activate()